# Construccion de Clasificacion de CIE-10 entre Agudos o Cronicos a traves de LLM (Gemini)

## Instalacion de dependencias

In [9]:
!pip install --upgrade google-cloud-aiplatform mdpd

In [14]:
!pip install pandas-gbq -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 4.1 MB/s eta 0:00:00a 0:00:01


In [76]:
!pip install -U db-dtypes

In [10]:
!pip install -U tabulate

## Cargar credenciales

In [6]:
import json
from google.oauth2 import service_account

with open('../../../../Downloads/sofia-data-305018-f51636353a22.json', 'rb') as f:
    credentials_dict = json.load(f)

# Create a credentials object from the dictionary
credentials = service_account.Credentials.from_service_account_info(credentials_dict)

## Importar tabla de CIE-10

In [7]:
from google.cloud import bigquery

client = bigquery.Client(project="sofia-data-305018", credentials=credentials)

In [13]:
import pandas_gbq

project_id = "sofia-data-305018"
query_cie_10_table = """
SELECT
  sub_code AS code, 
  SUBSTR(sub_code, 1, 1) AS letter,
  chapter,
  domain,
  name AS subdomain,
  definition,
FROM
  `sofia-data-305018.cadena_cuidado.icd10_2019` AS icd_table
--ON REGEXP_REPLACE(SPLIT(services_socio.services_diagnosis_icd, ',')[SAFE_OFFSET(0)], 'X$', '') = REGEXP_REPLACE(icd_table.sub_code, 'X$', '')

LEFT JOIN
  `sofia-data-305018.cadena_cuidado.icd10_head_code` AS icd_table_short
ON SUBSTR(icd_table.sub_code, 1, 3) = icd_table_short.head_code
;"""

df_cie_10 = pandas_gbq.read_gbq(
    query_cie_10_table,
    project_id=project_id,
    dialect="standard",
    credentials=credentials
)  
df_cie_10

Downloading: 100%|█████████████████████████████████████████████████████████████|


,code,letter,chapter,domain,subdomain,definition
0,A054,A,Chapter I - Certain infectious and parasitic d...,Intestinal infectious diseases (A00-A09),"Other bacterial foodborne intoxications, not e...",Foodborne Bacillus cereus intoxication
1,A021,A,Chapter I - Certain infectious and parasitic d...,Intestinal infectious diseases (A00-A09),Other salmonella infections,Salmonella sepsis
2,A014,A,Chapter I - Certain infectious and parasitic d...,Intestinal infectious diseases (A00-A09),Typhoid and paratyphoid fevers,"Paratyphoid fever, unspecified"
3,A001,A,Chapter I - Certain infectious and parasitic d...,Intestinal infectious diseases (A00-A09),Cholera,"Cholera due to Vibrio cholerae 01, biovar eltor"
4,A028,A,Chapter I - Certain infectious and parasitic d...,Intestinal infectious diseases (A00-A09),Other salmonella infections,Other specified salmonella infections
...,...,...,...,...,...,...
11238,Z831,Z,Chapter XXI - Factors influencing health statu...,Persons with potential health hazards related ...,Family history of other specific disorders,Family history of other infectious and parasit...
11239,Z82,Z,Chapter XXI - Factors influencing health statu...,Persons with potential health hazards related ...,Family history of certain disabilities and chr...,Family history of certain disabilities and chr...
11240,Z855,Z,Chapter XXI - Factors influencing health statu...,Persons with potential health hazards related ...,Personal history of malignant neoplasm,Personal history of malignant neoplasm of urin...
11241,Z871,Z,Chapter XXI - Factors influencing health statu...,Persons with potential health hazards related ...,Personal history of other diseases and conditions,Personal history of diseases of the digestive ...


## Prompt para Gemini

In [14]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

In [19]:
def generate(cie_10_letter: str, stream_response: bool):
  vertexai.init(project="sofia-data-305018", location="us-central1", credentials=credentials)

  generation_config = {
      "max_output_tokens": 8192,
      "temperature": 1.0,
      "top_p": 0.95,
  }

  safety_settings = {
      generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
      generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
      generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
      generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
  }

  textsi_1 = """As a Medical CIE-10 (ICD-10 or International Classification of Diseases 10th Revision) analysis expert, your role is to interpret CIE-10 codes with their respective diseases, offer an whole understanding about CIE-10 codes, context and description.

You understand the acute and chronic definitions like:
* Acute illnesses generally develop suddenly and last a short time, often only a few days or weeks.
* Acute conditions are often caused by a virus or an infection but can also be caused by an injury resulting from a fall or an automobile accident, or by the misuse of drugs or medications.
* Acute diseases come on rapidly, and are accompanied by distinct symptoms that require urgent or short-term care, and get better once they are treated. For example, a broken bone that might result from a fall must be treated by a doctor and will heal in time. Sometimes, an acute illness, such as the common cold, will just go away on its own. Most people with acute illnesses will soon recover. If hospitalized for an acute illness, after your condition improves, you’ll likely be eager to return to the comfort and familiarity of home. But being discharged from the hospital doesn’t mean you’re fully recovered. Learn more about measures can take to help hasten the return to your normal routine.
* Examples of acute medical conditions: Asthma attack, Broken bone, Bronchitis, Burn, COVID-19, Common cold, Flu, Heart attack, Pneumonia, Respiratory infection, Strep throat.
* Chronic conditions develop slowly and may worsen over an extended period of time—months to years.
* Chronic conditions are often caused by unhealthy behaviors that increase the risk of disease—poor nutrition, inadequate physical activity, overuse of alcohol, or smoking. Social, emotional, environmental, and genetic factors also play a role. As people age, they are more likely to develop one or more chronic conditions.
* Chronic conditions are slower to develop, may progress over time, and may have any number of warning signs or no signs at all. Common chronic conditions are arthritis, Alzheimer's disease, diabetes, heart disease, high blood pressure, and chronic kidney disease. Unlike acute conditions, chronic health conditions cannot be cured—only controlled. Living with chronic illness or managing the symptoms of a chronic condition can often be done by creating a health care plan in partnership with your physician—the plan may include taking medication, healthy eating, physical or occupational therapy, exercise, or complementary treatments, such as acupuncture or meditation. Frequently, chronic conditions can be prevented by practicing healthy lifestyle behaviors, such as staying physically active; maintaining a healthy weight and nutritional status; limiting sun exposure; and refraining from drugs, smoking, and excessive alcohol use.
* Examples of chronic medical conditions: Alzheimer’s disease, Arthritis, Chronic obstructive pulmonary disease (COPD), Depression, Diabetes, Heart disease, High blood pressure, High cholesterol, Obesity, Osteoporosis
* Acute conditions are severe and sudden in onset. This could describe anything from a broken bone to an asthma attack. A chronic condition, by contrast is a long-developing syndrome, such as osteoporosis or asthma. Note that osteoporosis, a chronic condition, may cause a broken bone, an acute condition. An acute asthma attack occurs in the midst of the chronic disease of asthma. Acute conditions, such as a first asthma attack, may lead to a chronic syndrome if untreated.

Examples of chronic diseases:
- Arthritis
- Bipolar Disorder
- Cancer
- Chronic Disease Update
- Chronic Respiratory Diseases
- Cardiovascular Disease
- Asthma
- Chronic Respiratory Diseases
- Chronic Obstructive Pulmonary Disease
- Sleep Apnea
- Mental Illness
- Dementia
- Depression
- Diabetes
- Fibromyalgia Syndrome
- Chronic Disease Data and Indicators
- Heart (Cardiovascular) Disease
- Hypertension
- Stroke
- Inflammatory bowel disease (IBD)
- Schizophrenia
- Neurological Conditions
- Epilepsy in Canada
- Multiple Sclerosis
- Parkinsonism, including Parkinson's disease: Highlights from the Canadian Chronic Disease Surveillance System
- Osteoporosis

To develop answers:

* Use clear and simple language, avoiding jargon and confusion.
* Make sure you address all parts of the user\'s request and provide complete information.
* Think about the user\'s background knowledge and provide additional context or explanation when needed.

Formatting and Language:

* You can understand both Spanish and English with high proficiency.
* Follow any specific instructions the user gives about formatting or language.
* Use proper formatting like tables to make complex data or results easier to understand.
* You output format is a table with these columns: cie_10_code,acute_probability,chronic_probability,avg_duration_time_days"""


  text1 = f"""Task: Read the markdown table below that it has the CIE-10 data in Spanish.
The markdown table has this schema: id,letter,grouping,description

id is CIE-10 code and letter is the CIE-10 code letter.

Markdown table in a CSV format:
{df_cie_10[df_cie_10.letter == cie_10_letter].to_csv(index=False)}
"""
  text2 = f"""Task: Classify medical diseases and conditions as either Acute or Chronic based on their CIE-10 codes and descriptions. 
In addition, estimate an average time (in days, could be more than 365 days) that the ICD-10 disease should last or how long you have to wait until you are evaluated again with a doctor since the disease was detected. 

Example 1:
Input:
code,letter,chapter,domain,subdomain,definition
J060,J,Chapter X - Diseases of the respiratory system (J00-J99),Acute upper respiratory infections (J00-J06),Acute upper respiratory infections of multiple and unspecified sites,Acute laryngopharyngitis
Output:
cie_10_code,acute_probability,chronic_probability,avg_duration_time_days
J060,0.9,0.1,14

Example 2:
Input:
code,letter,chapter,domain,subdomain,definition
J45,J,Chapter X - Diseases of the respiratory system (J00-J99),Chronic lower respiratory diseases (J40-J47),Asthma,Asthma
Output:
cie_10_code,acute_probability,chronic_probability,avg_duration_time_days
J45,0.2,0.8,None

Give me an answer using this format: cie_10_code,acute_probability,chronic_probability,avg_duration_time_days.
acute_probability and chronic_probability must sum 1.
avg_duration_time_days is None when the cie_10_code disease is chronic.
Ignores every word that is not part of the output table. Answer in a CSV format and include the headers.

Answer:
"""
  model = GenerativeModel(
    "gemini-1.5-pro-001",
    system_instruction=[textsi_1]
  )

  responses = model.generate_content(
      [text1, text2],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=stream_response,
  )
  return responses


responses = generate(cie_10_letter="A", stream_response=False)



In [21]:
print(responses.text)

cie_10_code,acute_probability,chronic_probability,avg_duration_time_days
A054,0.9,0.1,7
A021,0.9,0.1,14
A014,0.9,0.1,21
A001,0.9,0.1,10
A028,0.9,0.1,14
A042,0.9,0.1,7
A081,0.9,0.1,3
A08,0.9,0.1,7
A052,0.9,0.1,7
A032,0.9,0.1,7
A022,0.9,0.1,14
A043,0.9,0.1,7
A05,0.9,0.1,7
A099,0.9,0.1,7
A04,0.9,0.1,7
A068,0.1,0.9,None
A050,0.9,0.1,7
A02,0.9,0.1,14
A033,0.9,0.1,7
A067,0.1,0.9,None
A059,0.9,0.1,7
A01,0.9,0.1,21
A047,0.9,0.1,14
A061,0.1,0.9,None
A083,0.9,0.1,7
A082,0.9,0.1,7
A009,0.9,0.1,10
A065,0.1,0.9,None
A079,0.1,0.9,None
A046,0.9,0.1,7
A084,0.9,0.1,7
A053,0.9,0.1,7
A07,0.1,0.9,None
A066,0.1,0.9,None
A011,0.9,0.1,21
A039,0.9,0.1,7
A044,0.9,0.1,7
A058,0.9,0.1,7
A073,0.1,0.9,None
A085,0.9,0.1,7
A029,0.9,0.1,14
A041,0.9,0.1,7
A078,0.1,0.9,None
A060,0.9,0.1,7
A048,0.9,0.1,7
A038,0.9,0.1,7
A010,0.9,0.1,21
A072,0.1,0.9,None
A030,0.9,0.1,7
A040,0.9,0.1,7
A071,0.1,0.9,None
A090,0.9,0.1,7
A069,0.1,0.9,None
A09,0.9,0.1,7
A013,0.9,0.1,21
A012,0.9,0.1,21
A049,0.9,0.1,7
A051,0.9,0.1,30
A062,0.1,0.9,

In [22]:
cie_10_letter_list = [
    "A",
    "B",
    "C",
    "D",
    "E",
    "F",
    "G",
    "H",
    "I",
    "J",
    "K",
    "L",
    "M",
    "N",
    "O",
    "P",
    "Q",
    "R",
    "S",
    "T",
    "U",
    "V",
    "W",
    "X",
    "Y",
    "Z"
]

In [23]:
responses_list = []
for cie_10_letter in cie_10_letter_list:
  responses_list.append(generate(cie_10_letter=cie_10_letter, stream_response=False))

In [24]:
print(responses_list[0].text)

cie_10_code,acute_probability,chronic_probability,avg_duration_time_days
A054,0.9,0.1,7
A021,0.9,0.1,14
A014,0.9,0.1,21
A001,0.9,0.1,10
A028,0.9,0.1,14
A042,0.9,0.1,7
A081,0.9,0.1,7
A08,0.9,0.1,7
A052,0.9,0.1,7
A032,0.9,0.1,14
A022,0.9,0.1,14
A043,0.9,0.1,7
A05,0.9,0.1,7
A099,0.9,0.1,7
A04,0.9,0.1,7
A068,0.1,0.9,None
A050,0.9,0.1,7
A02,0.9,0.1,14
A033,0.9,0.1,14
A067,0.1,0.9,None
A059,0.9,0.1,7
A01,0.9,0.1,21
A047,0.9,0.1,14
A061,0.1,0.9,None
A083,0.9,0.1,7
A082,0.9,0.1,7
A009,0.9,0.1,10
A065,0.1,0.9,None
A079,0.1,0.9,None
A046,0.9,0.1,7
A084,0.9,0.1,7
A053,0.9,0.1,7
A07,0.1,0.9,None
A066,0.1,0.9,None
A011,0.9,0.1,21
A039,0.9,0.1,14
A044,0.9,0.1,7
A058,0.9,0.1,7
A073,0.1,0.9,None
A085,0.9,0.1,7
A029,0.9,0.1,14
A041,0.9,0.1,7
A078,0.1,0.9,None
A060,0.9,0.1,14
A048,0.9,0.1,7
A038,0.9,0.1,14
A010,0.9,0.1,21
A072,0.1,0.9,None
A030,0.9,0.1,14
A040,0.9,0.1,7
A071,0.1,0.9,None
A090,0.9,0.1,7
A069,0.1,0.9,None
A09,0.9,0.1,7
A013,0.9,0.1,21
A012,0.9,0.1,21
A049,0.9,0.1,7
A051,0.9,0.1,14
A062,0.

In [25]:
import io 
import pandas as pd

pd.read_csv(io.StringIO(responses_list[1].text.replace('```', '').replace('csv', '')), sep=",")

,cie_10_code,acute_probability,chronic_probability,avg_duration_time_days
0,B004,0.9,0.1,14.0
1,B084,0.9,0.1,14.0
2,B09,0.9,0.1,14.0
3,B028,0.9,0.1,21.0
4,B060,0.9,0.1,14.0
...,...,...,...,...
450,B960,0.1,0.9,NaN
451,B981,0.1,0.9,NaN
452,B975,0.1,0.9,NaN
453,B973,0.1,0.9,NaN


In [26]:
import io 
import pandas as pd

df_cie_10_with_class = pd.read_csv(io.StringIO(responses_list[0].text.replace('```', '').replace('csv', '')), sep=",")

for responses in responses_list[1:]:
    print(responses.text.replace('```', '').replace('csv', '')[0:60])
    df_cie_10_with_class = pd.concat([df_cie_10_with_class, pd.read_csv(io.StringIO(responses.text.replace('```', '').replace('csv', '')), sep=",")])
    

cie_10_code,acute_probability,chronic_probability,avg_durati
cie_10_code,acute_probability,chronic_probability,avg_durati
cie_10_code,acute_probability,chronic_probability,avg_durati
cie_10_code,acute_probability,chronic_probability,avg_durati
cie_10_code,acute_probability,chronic_probability,avg_durati
cie_10_code,acute_probability,chronic_probability,avg_durati
cie_10_code,acute_probability,chronic_probability,avg_durati
cie_10_code,acute_probability,chronic_probability,avg_durati
cie_10_code,acute_probability,chronic_probability,avg_durati
cie_10_code,acute_probability,chronic_probability,avg_durati
cie_10_code,acute_probability,chronic_probability,avg_durati
cie_10_code,acute_probability,chronic_probability,avg_durati
cie_10_code,acute_probability,chronic_probability,avg_durati
cie_10_code,acute_probability,chronic_probability,avg_durati
cie_10_code,acute_probability,chronic_probability,avg_durati
cie_10_code,acute_probability,chronic_probability,avg_durati
cie_10_code,acute_probab

In [17]:
#df_cie_10_with_class.rename(columns={"CIE-10 Code": "cie_10_code", "Acute Probability": "acute_probability", "Chronic Probability": "chronic_probability"}, inplace=True)

In [27]:
df_cie_10_with_class

,cie_10_code,acute_probability,chronic_probability,avg_duration_time_days
0,A054,0.9,0.1,7.0
1,A021,0.9,0.1,14.0
2,A014,0.9,0.1,21.0
3,A001,0.9,0.1,10.0
4,A028,0.9,0.1,14.0
...,...,...,...,...
544,Z906,0.0,1.0,NaN
545,Z914,0.0,1.0,NaN
546,Z93,0.0,1.0,NaN
547,Z875,0.0,1.0,NaN


In [28]:
import pandas_gbq

pandas_gbq.to_gbq(df_cie_10_with_class, 'cadena_cuidado.icd_10_2019_with_class_and_time', project_id='sofia-data-305018', if_exists='replace', credentials=credentials)


100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 5849.80it/s]


In [ ]:
SERVING_DOCKER_URI = ""

model_id = "microsoft/biogpt"
task="text-generation",

machine_type = "n1-standard-8"
accelerator_type = "NVIDIA_TESLA_V100"
accelerator_count = 1
max_length=200,
num_return_sequences=10,

# Fill the created service account.
service_account = ""

endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")

serving_env = { "BASE_MODEL_ID": model_id, "TASK": task, "MAX_LENGTH": max_length, "NUM_RETURN_SEQUENCES": num_return_sequences, }

artifact_uri = model_id if model_id.startswith("gs://") else None

model = aiplatform.Model.upload( display_name=model_name, serving_container_image_uri=SERVE_DOCKER_URI, serving_container_ports=[7080], serving_container_predict_route="/predictions/biogpt_serving", serving_container_health_route="/ping", serving_container_environment_variables=serving_env, artifact_uri=artifact_uri, )

model.deploy( endpoint=endpoint, machine_type=machine_type, accelerator_type=accelerator_type, accelerator_count=accelerator_count, deploy_request_timeout=1800, service_account=service_account, )

In [32]:
from typing import Dict, List, Union

from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    """
    `instances` can be either single instance of type dict or a list
    of instances.
    """
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances = instances if isinstance(instances, list) else [instances]
    instances = [
        json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
    ]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # The predictions are a google.protobuf.Value representation of the model's predictions.
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", prediction)

    return response


# [END aiplatform_predict_custom_trained_model_sample]

In [45]:
aiplatform.Endpoint.list(filter='display_name="biogpt-mg-one-click-deploy"')

 resource name: projects/892036853488/locations/us-central1/endpoints/9169889592256495616]

In [37]:
instances = [ {"prompt": "COVID-19 is"}, ]

response = endpoint.predict(instances=instances) 
print(response.predictions[0])

AttributeError: 'list' object has no attribute 'predict'

In [74]:
response = predict_custom_trained_model_sample(
    project="892036853488",
    endpoint_id="9169889592256495616",
    location="us-central1",
    instances={ "prompt": "Could you describe the ICD-10 A042 and Z966?", }
)

response
 deployed_model_id: 8146810515285344256
 prediction: Could you describe the ICD-10 A042 and Z966?


In [47]:
api_endpoint: str = "us-central1-aiplatform.googleapis.com"

client_options = {"api_endpoint": api_endpoint}
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

endpoint = client.endpoint_path(
        project='892036853488', location='us-central1', endpoint='9169889592256495616'
    )

In [49]:
instances = { "prompt": "COVID-19 is", }
print(instances)
instances = instances if isinstance(instances, list) else [instances]
print(instances)
instances = [
    json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
]
print(instances)

{'prompt': 'COVID-19 is'}
[{'prompt': 'COVID-19 is'}]
[struct_value {
  fields {
    key: "prompt"
    value {
      string_value: "COVID-19 is"
    }
  }
}
]


In [ ]:
instances.

In [51]:
response = client.predict(
        endpoint=endpoint, instances=instances
    )

In [53]:
print(response.metadata)

None


In [54]:
print(response.deployed_model_id)

8146810515285344256


In [55]:
print(response.model)

projects/892036853488/locations/us-central1/models/biogpt-1724887091443


In [56]:
{
  "instances": [
    { "prompt": "COVID-19 is"}
  ]
}

{'instances': [{'prompt': 'COVID-19 is'}]}

In [57]:
ENDPOINT_ID="9169889592256495616"
PROJECT_ID="892036853488"
INPUT_DATA_FILE="INPUT-JSON"

In [73]:
%%bash
ENDPOINT_ID="9169889592256495616"
PROJECT_ID="892036853488"

curl \
-X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
"https://us-central1-aiplatform.googleapis.com/v1/projects/$PROJECT_ID/locations/us-central1/endpoints/$ENDPOINT_ID:predict" \
-d '{
  "instances": [
    { "prompt": "Could you describe the ICD-10 A042 and Z966?"}
  ]
}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   378    0   290  100    88    151     45  0:00:01  0:00:01 --:--:--   197


{
  "predictions": [
    "Could you describe the ICD-10 A042 and Z966? (Table 1)."
  ],
  "deployedModelId": "8146810515285344256",
  "model": "projects/892036853488/locations/us-central1/models/biogpt-1724887091443",
  "modelDisplayName": "biogpt-1724887091443",
  "modelVersionId": "1"
}


In [75]:
a = {
 "predictions": [
   ".\n\n[User\nA042 -  Ambulatory surgical center.\nZ966 - non-material goods, materials used for the patient's care.\n\n**Explanation:**\n\nThe ICD-10 code A042 signifies that a medical procedure or service was performed in an Ambulatory Surgical Center (ASC). \n\nThe ICD-10 code Z966 signifies that the charges for the patient's care were related to non-material goods used for the patient's care.**\n\n**Key Takeaway:**\n\n* ASCs are healthcare facilities providing outpatient surgical services.\n*"
 ],
 "deployedModelId": "5503197534018863104",
 "model": "projects/892036853488/locations/us-central1/models/google_gemma2-2b-it-1724909157223",
 "modelDisplayName": "google_gemma2-2b-it-1724909157223",
 "modelVersionId": "1"
}

In [78]:
print(a['predictions'][0])

.

[User
A042 -  Ambulatory surgical center.
Z966 - non-material goods, materials used for the patient's care.

**Explanation:**

The ICD-10 code A042 signifies that a medical procedure or service was performed in an Ambulatory Surgical Center (ASC). 

The ICD-10 code Z966 signifies that the charges for the patient's care were related to non-material goods used for the patient's care.**

**Key Takeaway:**

* ASCs are healthcare facilities providing outpatient surgical services.
*
